In [1]:
import pandas as pd
from tqdm import tqdm
import re
import numpy as np

In [ ]:
# # Загрузка файла с компьютера
file_name = 'simulation_data.csv'
# Выбираем название столбца городов
city_column_name = 'город'
# Выбираем название столбца возрастов
city_column_age = 'возраст'
# Выбираем название столбца номеров
city_column_phone_number = 'номер телефона'


In [2]:
%%time
# Считывание данных в DataFrame
df = pd.read_csv(file_name)
# Вывод DataFrame
df.head()

CPU times: total: 375 ms
Wall time: 1.37 s


,город,номер телефона,пол,возраст,фамилия,имя,отчество,дата рождения,район,адрес,почта,whatsapp,telegram
0,Port Heatherstad,839.659.1933,Ж,90,Rodriguez,Evan,Walton,1945-08-20,North Danielfurt,"671 Cruz Rue\nNorth Carmen, AR 67148",patricia83@example.com,617.516.3957,cgillespie
1,New Ashleytown,2113417942,Ж,39,Frederick,Christine,Kennedy,2020-11-05,East Lisa,9730 Miranda Run Apt. 764\nWest Christophermou...,morganbarry@example.net,813-286-9334x8545,mariagregory
2,Smithland,001-531-302-7762x9317,М,66,Short,Janice,Hammond,1956-10-27,Port Jake,"9737 Mcconnell Cliff Suite 719\nAlyssastad, AZ...",jreed@example.net,+1-258-915-7389x08738,ypowell
3,Barberburgh,(375)935-5091,М,93,Leblanc,Cassandra,Perez,1917-02-28,Port Danaside,"32571 Bray Knolls\nNorth Heather, AK 05940",april68@example.com,387.247.8758,diana76
4,Christopherhaven,(969)867-5608x786,Ж,22,Gordon,Teresa,Buckley,1960-10-26,South Anthony,"788 Coleman Curve\nPort Arthur, IN 38138",sarah78@example.net,950-702-7959x7380,alexander52


In [4]:
%%time
# Определяем функцию для удаления скобок, тире и пробелов из номеров телефонов
def clean_phone_number(phone):
    # Удаление всех символов, кроме цифр
    phone = re.sub(r'\D', '', phone)
    return phone

# Применяем функцию к столбцу "номер телефона"
df[city_column_phone_number] = df[city_column_phone_number].apply(clean_phone_number)
df.head(2)

CPU times: total: 78.1 ms
Wall time: 162 ms


,город,номер телефона,пол,возраст,фамилия,имя,отчество,дата рождения,район,адрес,почта,whatsapp,telegram
0,Port Heatherstad,8396591933,Ж,90,Rodriguez,Evan,Walton,1945-08-20,North Danielfurt,"671 Cruz Rue\nNorth Carmen, AR 67148",patricia83@example.com,617.516.3957,cgillespie
1,New Ashleytown,2113417942,Ж,39,Frederick,Christine,Kennedy,2020-11-05,East Lisa,9730 Miranda Run Apt. 764\nWest Christophermou...,morganbarry@example.net,813-286-9334x8545,mariagregory


In [5]:
%%time
# Создаем два отдельных DataFrame
df_25_to_37 = df[(df[city_column_age] >= 25) & (df[city_column_age] <= 37)]
df_others = df[(df[city_column_age] < 25) | (df[city_column_age] > 37)]



CPU times: total: 15.6 ms
Wall time: 30.7 ms


In [6]:
%%time
# Группировка по городам для людей в возрасте от 25 до 37 лет
grouped_25_to_37 = df_25_to_37.groupby(city_column_name)

# Группировка по городам для всех остальных
grouped_others = df_others.groupby(city_column_name)


CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
import os
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Font

# Создание папок
folders = ["25_37", "Другие_возраста"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

def create_excel_file(dataframe, filename):
    # Создаем новую книгу Excel
    wb = Workbook()
    # Выбираем активный лист (первый лист по умолчанию)
    ws = wb.active
    
    # Преобразуем индексы столбцов в список
    headers = list(dataframe.columns)
    # Записываем заголовки
    ws.append(headers)

    # Записываем данные
    for row in dataframe.itertuples(index=False):
        ws.append(list(row))

    # Покрасим заголовки зеленым цветом и сделаем их жирными
    green_fill = PatternFill(start_color='bde0fe', end_color='bde0fe', fill_type='solid')
    bold_font = Font(bold=True)
    for cell in ws[1]:
        cell.fill = green_fill
        cell.font = bold_font

    # Добавляем границы для всех ячеек с данными
    border = Border(left=Side(style='thin'), 
                    right=Side(style='thin'), 
                    top=Side(style='thin'), 
                    bottom=Side(style='thin'))
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.border = border

    # Увеличиваем размер ячеек
    for col in ws.columns:
        max_length = 0
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2
        ws.column_dimensions[col[0].column_letter].width = adjusted_width

    # Сохраняем книгу Excel
    wb.save(filename)


# Функция для сохранения данных в Excel файлы по папкам и ограничению на количество строк
def save_to_excel(dataframe, folder_name):
    # Группировка данных по городам
    grouped_data = dataframe.groupby(city_column_name)
    
    # Итерация по каждой группе
    for city, group in grouped_data:
        # Создание подпапки для каждого города
        city_folder = os.path.join(folder_name, city)
        os.makedirs(city_folder, exist_ok=True)
        
        # Разделение данных на части по 50000 строк
        for i, chunk in enumerate(np.array_split(group, len(group) // 50000 + 1)):
            file_name = os.path.join(city_folder, f"{city}_{i+1}.xlsx")
            # Сохранение в кодировке cp1251
            create_excel_file(chunk, file_name)

# Сохранение данных для каждой возрастной категории
save_to_excel(df_25_to_37, "25_37")
save_to_excel(df_others, "Другие_возраста")


c:\Users\Bandos\Desktop\virt_env\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\Users\Bandos\Desktop\virt_env\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
